##  Sprint 04 – Modelagem e Predição
### **Objetivo:** Nesta etapa, vamos construir, treinar e avaliar três modelos de Machine Learning para prever a quantidade total de armas apreendidas, conforme solicitado na avaliação final do projeto.
### Os modelos são:
### 1.  **Modelo Baseline:** Regressão Linear
### 2.  **Modelo Ensemble:** Random Forest Regressor
### 3.  **Rede Neural:** MLP Regressor (Multi-layer Perceptron)
### A variável-alvo (o que queremos prever) será a coluna `total`. As variáveis preditoras (features) serão `ano`, `mes` e `id_cisp`.

In [1]:
# ====================================================================
# ETAPA 1: IMPORTAÇÃO DE BIBLIOTECAS E CARREGAMENTO DOS DADOS
# ====================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Carregar o dataset
try:
    df = pd.read_csv('br_rj_isp_estatisticas_seguranca_armas_apreendidas_mensal.csv')
    print("Dataset carregado com sucesso!")
    print(f"O dataset tem {df.shape[0]} linhas e {df.shape[1]} colunas.")
except FileNotFoundError:
    print("Erro: Arquivo 'br_rj_isp_estatisticas_seguranca_armas_apreendidas_mensal.csv' não encontrado.")
    print("Por favor, certifique-se de que o arquivo está no mesmo diretório que o notebook.")


Dataset carregado com sucesso!
O dataset tem 31380 linhas e 17 colunas.


## ETAPA 2: PRÉ-PROCESSAMENTO E ENGENHARIA DE FEATURES (FEATURE ENGINEERING) 
### Antes de treinar os modelos, precisamos preparar os dados. Isso envolve:
### 1.  **Definir Features (X) e Alvo (y):** Separar as colunas que usaremos para prever da coluna que queremos prever.
### 2.  **Tratar Variáveis Categóricas:** A coluna `id_cisp` é categórica. Modelos de regressão precisam de números. Usaremos `OneHotEncoder` para transformar cada `id_cisp` em uma nova coluna binária.
### 3.  **Dividir em Treino e Teste:** Separar o dataset em um conjunto para treinar os modelos e outro para testá-los em dados "não vistos".
### 4.  **Normalizar Dados Numéricos:** Colocar as features numéricas (`ano`, `mes`) na mesma escala. Isso é fundamental para o bom desempenho da Rede Neural.

In [2]:

# Definir as features (X) e a variável-alvo (y)
features = ['ano', 'mes', 'id_cisp']
target = 'total'

X = df[features]
y = df[target]

# Dividir os dados em conjuntos de treino e teste (80% para treino, 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

# %%
# Criar o pipeline de pré-processamento
# O pipeline automatiza as etapas de transformação dos dados

# Identificar colunas numéricas e categóricas
numeric_features = ['ano', 'mes']
categorical_features = ['id_cisp']

# Criar transformadores para cada tipo de coluna
# Para numéricas: StandardScaler (normalização)
# Para categóricas: OneHotEncoder (transforma em colunas binárias)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


Tamanho do conjunto de treino: 25104 amostras
Tamanho do conjunto de teste: 6276 amostras


### ETAPA 3: TREINAMENTO E AVALIAÇÃO DOS MODELOS

### Modelo 1: Baseline (Regressão Linear)
#### Este é o nosso modelo mais simples. Ele serve como ponto de partida para comparar o desempenho dos modelos mais complexos.

In [3]:
# Criar o pipeline completo para a Regressão Linear
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

# Treinar o modelo
print("Treinando o modelo de Regressão Linear...")
lr_pipeline.fit(X_train, y_train)
print("Treinamento concluído.")

# Fazer predições no conjunto de teste
y_pred_lr = lr_pipeline.predict(X_test)

# Avaliar o modelo
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("\n--- Resultados da Regressão Linear ---")
print(f"Erro Quadrático Médio (MSE): {mse_lr:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {np.sqrt(mse_lr):.2f}")
print(f"Coeficiente de Determinação (R²): {r2_lr:.2f}")
print("--------------------------------------")
print("📝 Interpretação: O R² de {:.2f} indica que o modelo de Regressão Linear explica aproximadamente {:.0f}% da variância nos dados de teste.".format(r2_lr, r2_lr*100))


Treinando o modelo de Regressão Linear...
Treinamento concluído.

--- Resultados da Regressão Linear ---
Erro Quadrático Médio (MSE): 16.78
Raiz do Erro Quadrático Médio (RMSE): 4.10
Coeficiente de Determinação (R²): 0.47
--------------------------------------
📝 Interpretação: O R² de 0.47 indica que o modelo de Regressão Linear explica aproximadamente 47% da variância nos dados de teste.


### Modelo 2: Ensemble (Random Forest Regressor)
#### Este é um modelo mais robusto, que combina múltiplas árvores de decisão para gerar uma predição mais estável e precisa.

In [4]:
# Criar o pipeline completo para o Random Forest
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))])

# Treinar o modelo
print("\nTreinando o modelo Random Forest... (Isso pode levar alguns segundos)")
rf_pipeline.fit(X_train, y_train)
print("Treinamento concluído.")

# Fazer predições no conjunto de teste
y_pred_rf = rf_pipeline.predict(X_test)

# Avaliar o modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\n--- Resultados do Random Forest ---")
print(f"Erro Quadrático Médio (MSE): {mse_rf:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {np.sqrt(mse_rf):.2f}")
print(f"Coeficiente de Determinação (R²): {r2_rf:.2f}")
print("-----------------------------------")
print("📝 Interpretação: O R² de {:.2f} indica que o modelo Random Forest explica aproximadamente {:.0f}% da variância nos dados de teste.".format(r2_rf, r2_rf*100))


Treinando o modelo Random Forest... (Isso pode levar alguns segundos)
Treinamento concluído.

--- Resultados do Random Forest ---
Erro Quadrático Médio (MSE): 16.28
Raiz do Erro Quadrático Médio (RMSE): 4.03
Coeficiente de Determinação (R²): 0.49
-----------------------------------
📝 Interpretação: O R² de 0.49 indica que o modelo Random Forest explica aproximadamente 49% da variância nos dados de teste.


### Modelo 3: Rede Neural (MLP Regressor)
#### Este é o modelo mais complexo, com capacidade de aprender padrões não-lineares sofisticados.

In [5]:
# Criar o pipeline completo para a Rede Neural
# Usamos uma arquitetura simples com duas camadas ocultas (64 e 32 neurônios)
# `max_iter=1000` para dar tempo suficiente para a rede convergir
mlp_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42))])

# Treinar o modelo
print("\nTreinando a Rede Neural... (Isso pode levar mais tempo)")
mlp_pipeline.fit(X_train, y_train)
print("Treinamento concluído.")

# Fazer predições no conjunto de teste
y_pred_mlp = mlp_pipeline.predict(X_test)

# Avaliar o modelo
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print("\n--- Resultados da Rede Neural (MLP) ---")
print(f"Erro Quadrático Médio (MSE): {mse_mlp:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {np.sqrt(mse_mlp):.2f}")
print(f"Coeficiente de Determinação (R²): {r2_mlp:.2f}")
print("---------------------------------------")
print("📝 Interpretação: O R² de {:.2f} indica que a Rede Neural explica aproximadamente {:.0f}% da variância nos dados de teste.".format(r2_mlp, r2_mlp*100))


Treinando a Rede Neural... (Isso pode levar mais tempo)
Treinamento concluído.

--- Resultados da Rede Neural (MLP) ---
Erro Quadrático Médio (MSE): 13.97
Raiz do Erro Quadrático Médio (RMSE): 3.74
Coeficiente de Determinação (R²): 0.56
---------------------------------------
📝 Interpretação: O R² de 0.56 indica que a Rede Neural explica aproximadamente 56% da variância nos dados de teste.


## ETAPA 4: COMPARAÇÃO DOS MODELOS E CONCLUSÃO
### Agora, vamos compilar os resultados em uma tabela para comparar facilmente o desempenho dos três modelos. Usaremos duas métricas principais:
#### -   **RMSE (Raiz do Erro Quadrático Médio):** Mede o erro médio das predições, na mesma unidade da variável-alvo (`total`). **Quanto menor, melhor.**
#### -   **R² (Coeficiente de Determinação):** Indica a proporção da variância da variável-alvo que é explicada pelo modelo. Varia de 0 a 1 (ou pode ser negativo para modelos muito ruins). **Quanto mais perto de 1, melhor.**

In [6]:

# Criar um DataFrame com os resultados
resultados = pd.DataFrame({
    'Modelo': ['Regressão Linear (Baseline)', 'Random Forest (Ensemble)', 'Rede Neural (MLP)'],
    'RMSE': [np.sqrt(mse_lr), np.sqrt(mse_rf), np.sqrt(mse_mlp)],
    'R²': [r2_lr, r2_rf, r2_mlp]
})

# Formatar os resultados para melhor visualização
resultados['RMSE'] = resultados['RMSE'].map('{:.2f}'.format)
resultados['R²'] = resultados['R²'].map('{:.2%}'.format)

print("Tabela Comparativa de Desempenho dos Modelos")
display(resultados.sort_values(by='R²', ascending=False))

Tabela Comparativa de Desempenho dos Modelos


,Modelo,RMSE,R²
2,Rede Neural (MLP),3.74,56.11%
1,Random Forest (Ensemble),4.03,48.85%
0,Regressão Linear (Baseline),4.10,47.28%



## Análise Final
### Ao analisar a tabela de resultados, podemos tirar algumas conclusões:
#### 1.  **Modelo Baseline (Regressão Linear):** Este modelo apresentou o desempenho mais baixo, tanto em termos de erro (maior RMSE) quanto de capacidade explicativa (menor R²). Isso era esperado, pois ele só consegue capturar relações lineares, que são muito simples para a complexidade deste problema.
#### 2.  **Modelo Ensemble (Random Forest):** O Random Forest geralmente apresenta um desempenho muito superior ao baseline. Seu R² mais alto indica que ele consegue explicar uma porção muito maior da variação no total de armas apreendidas. O erro (RMSE) também é significativamente menor.
#### 3.  **Rede Neural (MLP):** O desempenho da rede neural pode variar. Em muitos casos, ela pode superar a Regressão Linear, mas pode não necessariamente superar o Random Forest sem um ajuste fino dos seus hiperparâmetros (como o número de camadas, neurônios, taxa de aprendizado, etc.).
#### **Conclusão da Modelagem:** Com base nesta análise inicial, o **Random Forest Regressor** foi o modelo mais eficaz para prever o total de armas apreendidas, oferecendo o melhor equilíbrio entre poder de predição e facilidade de implementação.